In [22]:
import pandas as pd
import numpy as np
import torch
from transformers import pipeline

In [23]:
df = pd.read_pickle("reviews.pkl")

In [24]:
def preprocessing(df):
    df.drop(
        columns=[
            "images",
            "language",
            "url",
            "userId",
            "index",
            "restaurantId",
            "reviewId",
        ],
        inplace=True,
    )
    df["date"] = df["date"].apply(lambda x: x.split()[::2])
    mes = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12",
    }
    df["mes"] = df["date"].apply(lambda x: mes[x[1]])
    df["date"] = df[["date", "mes"]].apply(
        lambda x: str(x[0][0]) + "-" + str(x[1]) + "-" + str(x[0][2]), axis=1
    )
    df.drop(columns=["mes"], inplace=True)
    df["date"] = pd.to_datetime(df["date"])
    return df

In [25]:
df = preprocessing(df)

C:\Users\jilai\AppData\Local\Temp\ipykernel_18640\3951252518.py:34: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df["date"])


In [26]:
df.head(5)

,date,rating,text,title
0,2017-04-26,30,"Si quieres comer abundante, bien y sin floritu...",Comida casera muy casera
1,2015-08-17,40,cerveceria calsica muy cerquita de la plaza ma...,clasica cerveceria cons sabor autentico
2,2018-04-27,50,Una amiga que vive en alameda de Osuna juraba ...,Pasteles increíbles
3,2018-04-15,40,"la bolleria genial, el cafe bueno, el personal...",una buena merienda
4,2017-12-31,50,Impecable servicio y la mejor calidad y sabor ...,Excelente


In [27]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

c:\Users\jilai\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [28]:
def get_higher_sentiment_score(sentiment_scores):
    max = [(scores["score"], scores["label"]) for scores in sentiment_scores[0]]
    max.sort(reverse=True)
    return max[0][1]


In [38]:
df2 = df.copy().iloc[:1000]

In [39]:
df2["sentiment"] = df2["text"].apply(lambda x: get_higher_sentiment_score(distilled_student_sentiment_classifier(x)))

In [41]:
df2.to_pickle("reviews_with_sentiment.pkl")

In [7]:
classifier = pipeline(
    "zero-shot-classification",
    model="Recognai/bert-base-spanish-wwm-cased-xnli",
    framework="pt",
)

In [8]:
result = classifier(
    df["text"][6],
    candidate_labels=[
        "comida",
        "servicio al cliente",
        "ambiente",
        "limpieza",
        "precio",
        "ubicación",
        "bebida",
        "personal",
        "calidad",
    ],
    hypothesis_template="Se habla sobre {} del restaurante.",
    multi_label=True,
)
result

{'sequence': 'Las mejores tartas, pasteles, bolleria, pan de la zona. El de la zona de Bareco genial para tomar un café',
 'labels': ['calidad',
  'bebida',
  'comida',
  'servicio al cliente',
  'ubicación',
  'ambiente',
  'precio',
  'personal',
  'limpieza'],
 'scores': [0.9930290579795837,
  0.9789515137672424,
  0.9748647809028625,
  0.4981488883495331,
  0.36693307757377625,
  0.3639256954193115,
  0.2866499125957489,
  0.042407143861055374,
  0.026894304901361465]}